In [1]:
!pip install mediapipe opencv-python sounddevice librosa

In [2]:
# Core packages
import os, numpy as np, torch, torch.nn as nn, torch.optim as optim
from sklearn.model_selection import train_test_split
print("✅ Environment ready. CUDA:", torch.cuda.is_available(), "| Version:", torch.version.cuda)
os.makedirs("dataset", exist_ok=True)
os.makedirs("models", exist_ok=True)


✅ Environment ready. CUDA: True | Version: 11.8


In [4]:
import cv2, mediapipe as mp, time
GESTURES = ["up","down","left","right","forward","stop"]
SAVE_DIR = "dataset/gesture_npy"; os.makedirs(SAVE_DIR, exist_ok=True)
mp_hands = mp.solutions.hands.Hands(max_num_hands=1, min_detection_confidence=0.7)

def capture(label, n=100):
    cap=cv2.VideoCapture(0); collected=[]
    print(f"\n▶ Capturing '{label}' ({n} samples)"); time.sleep(2)
    while len(collected)<n:
        ret, frame=cap.read(); 
        if not ret: break
        rgb=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        res=mp_hands.process(rgb)
        if res.multi_hand_landmarks:
            pts=np.array([[p.x,p.y,p.z] for p in res.multi_hand_landmarks[0].landmark]).flatten()
            collected.append(pts)
            cv2.putText(frame,f"{label}:{len(collected)}/{n}",(10,40),1,1.5,(0,255,0),2)
        cv2.imshow("Gesture",frame)
        if cv2.waitKey(1)&0xFF==27: break
    cap.release(); cv2.destroyAllWindows()
    np.save(f"{SAVE_DIR}/{label}.npy",np.array(collected))
    print(f"✅ Saved {len(collected)} samples for '{label}'")

for g in GESTURES: capture(g)



▶ Capturing 'up' (100 samples)
✅ Saved 100 samples for 'up'

▶ Capturing 'down' (100 samples)
✅ Saved 100 samples for 'down'

▶ Capturing 'left' (100 samples)
✅ Saved 100 samples for 'left'

▶ Capturing 'right' (100 samples)
✅ Saved 100 samples for 'right'

▶ Capturing 'forward' (100 samples)
✅ Saved 100 samples for 'forward'

▶ Capturing 'stop' (100 samples)
✅ Saved 100 samples for 'stop'


In [5]:
import glob
files=glob.glob("dataset/gesture_npy/*.npy")
X=[]; y=[]
for idx,f in enumerate(files):
    data=np.load(f); X.append(data); y.extend([idx]*len(data))
X=np.vstack(X); y=np.array(y)
np.save("dataset/gesture_landmarks.npy",X)
np.save("dataset/gesture_labels.npy",y)
print("✅ Combined gesture dataset:", X.shape, "Labels:", len(np.unique(y)))


✅ Combined gesture dataset: (600, 63) Labels: 6


In [4]:
import torchaudio
from torchaudio.datasets import SPEECHCOMMANDS
WORDS=["up","down","left","right","go","stop"]
SAVE="dataset/voice_npy"; os.makedirs(SAVE,exist_ok=True)
ds=SPEECHCOMMANDS(root="./",download=True)
X=[]; y=[]
for idx,w in enumerate(WORDS):
    for wave,sr,label,*_ in ds:
        if label!=w: continue
        mfcc=torchaudio.transforms.MFCC(n_mfcc=40)(wave)
        X.append(mfcc.mean(dim=2).squeeze().numpy()); y.append(idx)
X=np.array(X); y=np.array(y)
np.save(f"{SAVE}/voice_mfcc.npy",X); np.save(f"{SAVE}/voice_labels.npy",y)
print("✅ Voice dataset:", X.shape)


100%|██████████| 2.26G/2.26G [11:08<00:00, 3.63MB/s] 
c:\Users\Abhishek\anaconda3\envs\drone_ai\Lib\site-packages\torchaudio\functional\functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


✅ Voice dataset: (22971, 40)


In [6]:
import pandas as pd
fer=pd.read_csv(r"C:\Users\Abhishek\Desktop\Abhishek\7th sem\Auto drones\prj\fer2013.csv\fer2013.csv")  # download from Kaggle beforehand
X=[]; y=[]
for _,r in fer.iterrows():
    if r["emotion"]==3: label=1
    elif r["emotion"] in [0,1,2,4,5,6]: label=0
    else: continue
    img=np.array(list(map(int,r["pixels"].split()))).reshape(48,48)/255.0
    X.append(img); y.append(label)
X=np.array(X)[:,None,:,:]; y=np.array(y)
np.save("dataset/faces.npy",X); np.save("dataset/labels.npy",y)
print("✅ Emotion dataset:", X.shape, "Labels:", np.unique(y,return_counts=True))


✅ Emotion dataset: (35887, 1, 48, 48) Labels: (array([0, 1]), array([26898,  8989], dtype=int64))


In [9]:
import torch, torch.nn as nn, torch.optim as optim, numpy as np
from sklearn.model_selection import train_test_split

# Load dataset
X = np.load("dataset/gesture_landmarks.npy")
y = np.load("dataset/gesture_labels.npy")

# Split and convert to tensors
Xtr, Xv, ytr, yv = train_test_split(X, y, test_size=0.2)
Xtr, ytr = torch.tensor(Xtr, dtype=torch.float32), torch.tensor(ytr, dtype=torch.long)
Xv,  yv  = torch.tensor(Xv,  dtype=torch.float32), torch.tensor(yv,  dtype=torch.long)

# Model
class GestureNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(63, 64), nn.ReLU(),
            nn.Linear(64, 32), nn.ReLU(),
            nn.Linear(32, 6)
        )
    def forward(self, x): return self.fc(x)

m = GestureNet().cuda()
opt = optim.Adam(m.parameters(), lr=1e-3)
crit = nn.CrossEntropyLoss()

# Training loop
for e in range(80):
    opt.zero_grad()
    out = m(Xtr.cuda())
    loss = crit(out, ytr.cuda())
    loss.backward()
    opt.step()
    if e % 10 == 0:
        acc = (m(Xv.cuda()).argmax(1) == yv.cuda()).float().mean()
        print(f"Ep{e}: loss={loss.item():.3f}, acc={acc:.3f}")

torch.save(m.state_dict(), "models/gesture_model.pth")
print("✅ gesture_model.pth saved.")


Ep0: loss=1.799, acc=0.158
Ep10: loss=1.776, acc=0.158
Ep20: loss=1.753, acc=0.658
Ep30: loss=1.717, acc=0.658
Ep40: loss=1.656, acc=0.817
Ep50: loss=1.564, acc=0.850
Ep60: loss=1.437, acc=0.758
Ep70: loss=1.282, acc=0.967
✅ gesture_model.pth saved.


In [42]:
# ============================================================
# 🎤 Train Voice Command Model (Fully Connected)
# Works for dataset/voice_npy/voice_mfcc.npy (shape: N x 40)
# ============================================================

import torch, torch.nn as nn, numpy as np, os
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

# ------------------------------------------------------------
# 1️⃣ Load MFCC & Labels
# ------------------------------------------------------------
X = np.load("dataset/voice_npy/voice_mfcc.npy")   # shape (N, 40)
y = np.load("dataset/voice_npy/voice_labels.npy") # shape (N,)

print(f"✅ Loaded MFCCs: {X.shape}, Labels: {y.shape}, Classes: {np.unique(y)}")

# Normalize features sample-wise
X = (X - X.mean(axis=0)) / (X.std(axis=0) + 1e-6)

# Convert to tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

# Split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

train_dl = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
val_dl   = DataLoader(TensorDataset(X_val, y_val), batch_size=128)

# ------------------------------------------------------------
# 2️⃣ Define Model (simple dense network)
# ------------------------------------------------------------
class VoiceFC(nn.Module):
    def __init__(self, n_classes=6):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(40, 128), nn.BatchNorm1d(128), nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, n_classes)
        )
    def forward(self, x): return self.net(x)

# ------------------------------------------------------------
# 3️⃣ Train Setup
# ------------------------------------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
model = VoiceFC(n_classes=len(torch.unique(y))).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
crit = nn.CrossEntropyLoss()

# ------------------------------------------------------------
# 4️⃣ Training Loop
# ------------------------------------------------------------
EPOCHS = 25
best_acc = 0
for epoch in range(EPOCHS):
    model.train(); total_loss = 0
    for xb, yb in train_dl:
        xb, yb = xb.to(device), yb.to(device)
        opt.zero_grad()
        out = model(xb)
        loss = crit(out, yb)
        loss.backward(); opt.step()
        total_loss += loss.item() * xb.size(0)
    total_loss /= len(train_dl.dataset)

    # Validation
    model.eval(); correct = 0
    with torch.no_grad():
        for xb, yb in val_dl:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb).argmax(1)
            correct += (preds == yb).sum().item()
    acc = correct / len(X_val)
    print(f"Epoch {epoch+1:02d}: loss={total_loss:.4f}, val_acc={acc:.3f}")

    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "voice_model.pth")

print(f"🏁 Training complete. Best val_acc={best_acc:.3f}")
print("📁 Saved model → voice_model.pth")


✅ Loaded MFCCs: (22971, 40), Labels: (22971,), Classes: [0 1 2 3 4 5]
Epoch 01: loss=1.4133, val_acc=0.528
Epoch 02: loss=1.2639, val_acc=0.535
Epoch 03: loss=1.2269, val_acc=0.556
Epoch 04: loss=1.1984, val_acc=0.558
Epoch 05: loss=1.1760, val_acc=0.565
Epoch 06: loss=1.1589, val_acc=0.572
Epoch 07: loss=1.1537, val_acc=0.578
Epoch 08: loss=1.1373, val_acc=0.579
Epoch 09: loss=1.1173, val_acc=0.582
Epoch 10: loss=1.1179, val_acc=0.587
Epoch 11: loss=1.1083, val_acc=0.579
Epoch 12: loss=1.0970, val_acc=0.577
Epoch 13: loss=1.0987, val_acc=0.584
Epoch 14: loss=1.0814, val_acc=0.586
Epoch 15: loss=1.0762, val_acc=0.585
Epoch 16: loss=1.0718, val_acc=0.589
Epoch 17: loss=1.0604, val_acc=0.593
Epoch 18: loss=1.0636, val_acc=0.588
Epoch 19: loss=1.0539, val_acc=0.594
Epoch 20: loss=1.0508, val_acc=0.594
Epoch 21: loss=1.0374, val_acc=0.592
Epoch 22: loss=1.0345, val_acc=0.597
Epoch 23: loss=1.0316, val_acc=0.598
Epoch 24: loss=1.0247, val_acc=0.599
Epoch 25: loss=1.0231, val_acc=0.593
🏁 Tra

In [33]:
import torch, torch.nn as nn, torch.optim as optim, numpy as np
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

# Load data
X = np.load("dataset/faces.npy")   # (N,1,48,48)
y = np.load("dataset/labels.npy")

# Split
Xtr, Xv, ytr, yv = train_test_split(X, y, test_size=0.2, random_state=42)
Xtr = torch.tensor(Xtr, dtype=torch.float32)
ytr = torch.tensor(ytr, dtype=torch.long)
Xv  = torch.tensor(Xv,  dtype=torch.float32)
yv  = torch.tensor(yv,  dtype=torch.long)

# Dataloaders  ✅
BATCH_SIZE = 128
train_loader = DataLoader(TensorDataset(Xtr, ytr), batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(TensorDataset(Xv, yv), batch_size=BATCH_SIZE)

# Model ✅ (pooled + fixed linear dim)
class EmotionNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 8, 3, 1), nn.ReLU(),
            nn.MaxPool2d(2, 2),               # 48→23
            nn.Conv2d(8, 16, 3, 1), nn.ReLU(),
            nn.MaxPool2d(2, 2),               # 21→10
            nn.Flatten(),
            nn.Linear(16 * 10 * 10, 2)
        )
    def forward(self, x): return self.conv(x)

device = "cuda" if torch.cuda.is_available() else "cpu"
m = EmotionNet().to(device)
opt = optim.Adam(m.parameters(), lr=1e-3)
crit = nn.CrossEntropyLoss()

# Training loop ✅ (mini-batches)
for e in range(15):
    m.train()
    total_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        opt.zero_grad()
        out = m(xb)
        loss = crit(out, yb)
        loss.backward()
        opt.step()
        total_loss += loss.item() * xb.size(0)
    total_loss /= len(train_loader.dataset)

    # Validation accuracy
    m.eval(); correct = 0; total = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            preds = m(xb.to(device)).argmax(1).cpu()
            correct += (preds == yb).sum().item()
            total += yb.size(0)
    acc = correct / total
    print(f"Ep{e:02d}: loss={total_loss:.3f}, acc={acc:.3f}")

torch.save(m.state_dict(), "models/emotion_model.pth")
print("✅ emotion_model.pth saved.")


Ep00: loss=0.523, acc=0.789
Ep01: loss=0.455, acc=0.794
Ep02: loss=0.439, acc=0.799
Ep03: loss=0.426, acc=0.812
Ep04: loss=0.413, acc=0.826
Ep05: loss=0.400, acc=0.824
Ep06: loss=0.390, acc=0.836
Ep07: loss=0.381, acc=0.826
Ep08: loss=0.375, acc=0.837
Ep09: loss=0.367, acc=0.829
Ep10: loss=0.363, acc=0.845
Ep11: loss=0.358, acc=0.845
Ep12: loss=0.353, acc=0.844
Ep13: loss=0.349, acc=0.844
Ep14: loss=0.344, acc=0.850
✅ emotion_model.pth saved.


In [12]:
import time

g=GestureNet(); g.load_state_dict(torch.load("models/gesture_model.pth",map_location="cpu")); g.eval()
v=VoiceNet();   v.load_state_dict(torch.load("models/voice_model.pth",map_location="cpu"));   v.eval()
e=EmotionNet(); e.load_state_dict(torch.load("models/emotion_model.pth",map_location="cpu")); e.eval()

ACTIONS=["stop","up","down","left","right","forward"]

while True:
    # Replace random data with live inputs from sensors/mic/camera
    landmarks=np.random.rand(63).astype(np.float32)
    mfcc=np.random.rand(40).astype(np.float32)
    face=np.random.rand(1,48,48).astype(np.float32)

    emo=torch.argmax(e(torch.tensor(face).unsqueeze(0))).item()
    if emo==1:
        cmd="land"
    else:
        gest=torch.argmax(g(torch.tensor(landmarks).unsqueeze(0))).item()
        if gest!=0:
            cmd=ACTIONS[gest]
        else:
            voc=torch.argmax(v(torch.tensor(mfcc).unsqueeze(0))).item()
            cmd=ACTIONS[voc]

    print(f"🚀 Command: {cmd}")
    time.sleep(1)


C:\Users\Abhishek\AppData\Local\Temp\ipykernel_25992\4085414792.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  g=GestureNet(); g.load_state_dict(torch.load("models/gest

🚀 Command: right
🚀 Command: right
🚀 Command: right
🚀 Command: right
🚀 Command: right


KeyboardInterrupt: 

In [ ]:
import os, time, json, socket, warnings, collections
warnings.filterwarnings("ignore", category=UserWarning)

import cv2
import numpy as np
import torch
import torch.nn as nn
import torchaudio
import sounddevice as sd
import mediapipe as mp
from sklearn.neighbors import KNeighborsClassifier


UDP_IP, UDP_PORT = "127.0.0.1", 5501
SAMPLE_RATE = 16000
VOICE_N_MFCC = 40
VAD_RMS_THRESH = 0.0015
CYCLE_SECS = 1.0
SURPRISE_THRESH = 0.15
GESTURE_SMOOTH = 3
DEBUG = True


sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
print(f"📡 UDP socket ready (IP={UDP_IP}, PORT={UDP_PORT})")


def get_camera():
    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    if not cap.isOpened():
        raise RuntimeError(" Could not open webcam.")
    print("📹 Camera started successfully.")
    return cap

mp_hands = mp.solutions.hands
face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

_mfcc_transform = torchaudio.transforms.MFCC(
    sample_rate=SAMPLE_RATE,
    n_mfcc=VOICE_N_MFCC,
    melkwargs={"n_fft": 512, "hop_length": 160, "n_mels": VOICE_N_MFCC, "center": False}
)

def record_audio_get_mfcc(duration=1.0):
    sd.default.reset()
    audio = sd.rec(int(duration * SAMPLE_RATE), samplerate=SAMPLE_RATE, channels=1, dtype='float32')
    sd.wait()
    rms = float(np.sqrt(np.mean(np.square(audio))))
    if rms < VAD_RMS_THRESH:
        if DEBUG: print(f" Silence (RMS={rms:.5f})")
        return None
    waveform = torch.tensor(audio.T)
    mfcc = _mfcc_transform(waveform).mean(dim=2)
    mfcc = (mfcc - mfcc.mean()) / (mfcc.std() + 1e-6)
    return mfcc

class VoiceNet_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(1,8,3,padding=1), nn.ReLU(),
            nn.Conv1d(8,16,3,padding=1), nn.ReLU(),
            nn.MaxPool1d(2)
        )
        self.fc = nn.Sequential(
            nn.Linear(320,64), nn.ReLU(),
            nn.Linear(64,6)
        )
    def forward(self,x):
        if len(x.shape)==2: x = x.unsqueeze(1)
        x = self.cnn(x)
        x = x.view(x.size(0),-1)
        return self.fc(x)

class EmotionNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1,8,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(8,16,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16,32,3,padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Linear(32*6*6,128), nn.ReLU(),
            nn.Linear(128,7)
        )
    def forward(self,x):
        x = self.conv(x)
        x = x.view(x.size(0),-1)
        return self.fc(x)


def safe_load(model, path, name):
    try:
        model.load_state_dict(torch.load(path, map_location="cpu"), strict=False)
        print(f" {name} model loaded.")
    except Exception as e:
        print(f" Could not load {name} model: {e}")

voice_model, emotion_model = VoiceNet_CNN(), EmotionNet()
safe_load(voice_model, "models/voice_model.pth", "Voice")
safe_load(emotion_model, "models/emotion_model.pth", "Emotion")
voice_model.eval(); emotion_model.eval()

ACTIONS = ["stop", "up", "down", "left", "right", "forward"]
EMOTIONS = ["angry", "disgust", "fear", "happy", "sad", "surprise", "neutral"]


def load_gesture_data():
    gesture_paths = {
        "up": "dataset/gesture_npy/up.npy" if os.path.exists("dataset/gesture_npy/up.npy") else "/mnt/data/up.npy",
        "down": "dataset/gesture_npy/down.npy" if os.path.exists("dataset/gesture_npy/down.npy") else "/mnt/data/down.npy",
        "left": "dataset/gesture_npy/left.npy" if os.path.exists("dataset/gesture_npy/left.npy") else "/mnt/data/left.npy",
        "right": "dataset/gesture_npy/right.npy" if os.path.exists("dataset/gesture_npy/right.npy") else "/mnt/data/right.npy",
        "forward": "dataset/gesture_npy/forward.npy" if os.path.exists("dataset/gesture_npy/forward.npy") else "/mnt/data/forward.npy",
        "stop": "dataset/gesture_npy/stop.npy" if os.path.exists("dataset/gesture_npy/stop.npy") else "/mnt/data/stop.npy",
    }

    X, y = [], []
    for label, path in gesture_paths.items():
        arr = np.load(path)
        X.append(arr)
        y += [label] * arr.shape[0]
    X = np.vstack(X)
    y = np.array(y)
    print(f" Loaded gesture data: {X.shape} samples, {len(set(y))} classes.")
    return X, y

X_gest, y_gest = load_gesture_data()
gesture_clf = KNeighborsClassifier(n_neighbors=3)
gesture_clf.fit(X_gest, y_gest)
gesture_hist = collections.deque(maxlen=GESTURE_SMOOTH)


def get_gesture_cmd(frame):
    with mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7) as hands:
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        res = hands.process(rgb)
        if not res.multi_hand_landmarks:
            gesture_hist.append("stop")
            return "stop", 0.0

        lm = np.array([[lm.x, lm.y, lm.z] for lm in res.multi_hand_landmarks[0].landmark])
        lm_min, lm_max = lm.min(axis=0), lm.max(axis=0)
        norm = (lm - lm_min) / (lm_max - lm_min + 1e-6)
        lm_flat = norm.flatten().reshape(1, -1)

        cmd = gesture_clf.predict(lm_flat)[0]
        probs = gesture_clf.predict_proba(lm_flat)[0]
        conf = np.max(probs)
        gesture_hist.append(cmd)
        smooth = max(set(gesture_hist), key=gesture_hist.count)
        if DEBUG: print(f" Gesture → {smooth} (conf={conf:.2f}) | {dict(zip(gesture_clf.classes_, np.round(probs,2)))}")
        return smooth, conf

def get_voice_cmd(mfcc):
    if mfcc is None: return None, 0.0
    with torch.no_grad():
        out = voice_model(mfcc)
        probs = torch.softmax(out, dim=1).numpy().flatten()
    cmd = ACTIONS[int(np.argmax(probs))]
    conf = float(np.max(probs))
    if DEBUG: print(f" Voice → {cmd} (conf={conf:.2f}) | {np.round(probs,2)}")
    return cmd, conf

def get_emotion_cmd(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.1, 5)
    if len(faces) == 0:
        return "none", 0.0
    (x, y, w, h) = faces[0]
    face = gray[y:y+h, x:x+w]
    face = cv2.equalizeHist(face)
    face = cv2.resize(face, (48, 48))
    face_t = torch.tensor(face, dtype=torch.float32).unsqueeze(0).unsqueeze(0) / 255.0

    with torch.no_grad():
        out = emotion_model(face_t)
        probs = torch.softmax(out, dim=1).numpy().flatten()

    emotion = EMOTIONS[int(np.argmax(probs))]
    conf = float(np.max(probs))

    if DEBUG:
        print(f" Emotions → {dict(zip(EMOTIONS, [round(p,2) for p in probs]))}")

    if emotion == "surprise" and conf >= SURPRISE_THRESH:
        print(f" Surprise confirmed (conf={conf:.2f}) → LAND")
        return "land", conf

    return emotion, conf

# ======================================================
#  MAIN LOOP
# ======================================================
def run():
    cap = get_camera()
    last_cycle = 0
    try:
        while True:
            ret, frame = cap.read()
            if not ret: break

            # Emotion check every frame
            emo_cmd, e_conf = get_emotion_cmd(frame)
            if emo_cmd == "land":
                payload = {"source": "face", "command": "land", "timestamp": time.strftime("%H:%M:%S")}
                sock.sendto(json.dumps(payload).encode(), (UDP_IP, UDP_PORT))
                print(" LAND triggered → shutting down.")
                break

            # Gesture & Voice every cycle
            if time.time() - last_cycle > CYCLE_SECS:
                last_cycle = time.time()

                gesture_cmd, _ = get_gesture_cmd(frame)
                mfcc = record_audio_get_mfcc(0.8)
                voice_cmd, _ = get_voice_cmd(mfcc)
                sd.default.reset()

                # Priority: gesture > voice > hover
                if gesture_cmd != "stop":
                    final, src = gesture_cmd, "camera"
                elif voice_cmd not in [None, "stop"]:
                    final, src = voice_cmd, "mic"
                else:
                    final, src = "hover", "system"

                payload = {"source": src, "command": final, "timestamp": time.strftime("%H:%M:%S")}
                sock.sendto(json.dumps(payload).encode(), (UDP_IP, UDP_PORT))
                print(f" Final: [{src.upper()}] → {final.upper()}")

            # HUD overlay
            overlay = frame.copy()
            cv2.putText(overlay, f"Emotion: {emo_cmd} ({e_conf:.2f})", (10,30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
            cv2.imshow("Drone Feed", overlay)

            if cv2.waitKey(1) & 0xFF == ord('q'): break

    finally:
        cap.release(); cv2.destroyAllWindows(); sock.close()
        print(" Shutdown complete.")

if __name__ == "__main__":
    run()


📡 UDP socket ready (IP=127.0.0.1, PORT=5501)
 Voice model loaded.
 Emotion model loaded.
 Loaded gesture data: (600, 63) samples, 6 classes.


C:\Users\Abhishek\AppData\Local\Temp\ipykernel_28884\3190919987.py:95: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path, map_location="cpu

📹 Camera started successfully.
 Emotions → {'angry': 0.15, 'disgust': 0.15, 'fear': 0.16, 'happy': 0.13, 'sad': 0.14, 'surprise': 0.13, 'neutral': 0.14}
 Silence (RMS=0.00001)
 Final: [SYSTEM] → HOVER
 Emotions → {'angry': 0.15, 'disgust': 0.15, 'fear': 0.16, 'happy': 0.13, 'sad': 0.14, 'surprise': 0.13, 'neutral': 0.14}
 Silence (RMS=0.00001)
 Final: [SYSTEM] → HOVER
 Emotions → {'angry': 0.15, 'disgust': 0.15, 'fear': 0.16, 'happy': 0.13, 'sad': 0.14, 'surprise': 0.13, 'neutral': 0.14}
 Emotions → {'angry': 0.15, 'disgust': 0.15, 'fear': 0.16, 'happy': 0.13, 'sad': 0.14, 'surprise': 0.13, 'neutral': 0.14}
 Silence (RMS=0.00001)
 Final: [SYSTEM] → HOVER
 Emotions → {'angry': 0.15, 'disgust': 0.15, 'fear': 0.16, 'happy': 0.13, 'sad': 0.14, 'surprise': 0.13, 'neutral': 0.14}
 Emotions → {'angry': 0.15, 'disgust': 0.15, 'fear': 0.16, 'happy': 0.13, 'sad': 0.14, 'surprise': 0.13, 'neutral': 0.14}
 Silence (RMS=0.00001)
 Final: [SYSTEM] → HOVER
 Emotions → {'angry': 0.15, 'disgust': 0.15, 